In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('datasets/Classification/KNNDataset.csv')

In [3]:
data.rename(columns = {"diagnosis": "Class"}, inplace=True)
data.dropna(axis=1, inplace=True)

In [4]:
clss = data["Class"].unique()
print(clss)

['M' 'B']


In [5]:
data.drop(columns = ['id'], inplace=True)

In [6]:
cols = {}
cols['Class'] = {val : index for index, val in enumerate(data['Class'].unique())}
data['Class'] = [cols['Class'][val] for val in data['Class']]
cols

{'Class': {'M': 0, 'B': 1}}

In [7]:
data

Class  radius_mean  texture_mean  perimeter_mean  area_mean  \
0        0        17.99         10.38          122.80     1001.0   
1        0        20.57         17.77          132.90     1326.0   
2        0        19.69         21.25          130.00     1203.0   
3        0        11.42         20.38           77.58      386.1   
4        0        20.29         14.34          135.10     1297.0   
..     ...          ...           ...             ...        ...   
564      0        21.56         22.39          142.00     1479.0   
565      0        20.13         28.25          131.20     1261.0   
566      0        16.60         28.08          108.30      858.1   
567      0        20.60         29.33          140.10     1265.0   
568      1         7.76         24.54           47.92      181.0   

     smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0            0.11840           0.27760         0.30010              0.14710   
1            0.08474           0.07864         0.08690              0.07017   
2            0.10960           0.15990         0.19740              0.12790   
3            0.14250           0.28390         0.24140              0.10520   
4            0.10030           0.13280         0.19800              0.10430   
..               ...               ...             ...                  ...   
564          0.11100           0.11590         0.24390              0.13890   
565          0.09780           0.10340         0.14400              0.09791   
566          0.08455           0.10230         0.09251              0.05302   
567          0.11780           0.27700         0.35140              0.15200   
568          0.05263           0.04362         0.00000              0.00000   

     symmetry_mean  ...  radius_worst  texture_worst  perimeter_worst  \
0           0.2419  ...        25.380          17.33           184.60   
1           0.1812  ...        24.990          23.41           158.80   
2           0.2069  ...        23.570          25.53           152.50   
3           0.2597  ...        14.910          26.50            98.87   
4           0.1809  ...        22.540          16.67           152.20   
..             ...  ...           ...            ...              ...   
564         0.1726  ...        25.450          26.40           166.10   
565         0.1752  ...        23.690          38.25           155.00   
566         0.1590  ...        18.980          34.12           126.70   
567         0.2397  ...        25.740          39.42           184.60   
568         0.1587  ...         9.456          30.37            59.16   

     area_worst  smoothness_worst  compactness_worst  concavity_worst  \
0        2019.0           0.16220            0.66560           0.7119   
1        1956.0           0.12380            0.18660           0.2416   
2        1709.0           0.14440            0.42450           0.4504   
3         567.7           0.20980            0.86630           0.6869   
4        1575.0           0.13740            0.20500           0.4000   
..          ...               ...                ...              ...   
564      2027.0           0.14100            0.21130           0.4107   
565      1731.0           0.11660            0.19220           0.3215   
566      1124.0           0.11390            0.30940           0.3403   
567      1821.0           0.16500            0.86810           0.9387   
568       268.6           0.08996            0.06444           0.0000   

     concave points_worst  symmetry_worst  fractal_dimension_worst  
0                  0.2654          0.4601                  0.11890  
1                  0.1860          0.2750                  0.08902  
2                  0.2430          0.3613                  0.08758  
3                  0.2575          0.6638                  0.17300  
4                  0.1625          0.2364                  0.07678  
..                    ...             ...                      ...  
56

In [8]:
data.columns

Index(['Class', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [9]:
trainData = data.sample(frac=0.8, random_state=0)

In [10]:
testData = data.drop(trainData.index)

In [11]:
def costFunction(x,y,w):
    m = len(x)
    mat = np.matmul(x,w)
    mat = np.subtract(mat,y)
    # print(mat)
    J = 1/(1 + np.exp((-1)*np.clip(mat, -500, 500))).astype(np.longdouble)
    return sum(J)

In [12]:
def gradient(x,y,w,b,learning_rate,iterations=10000,stoping_threshold=0.000000000001):
    x = x.to_numpy()
    y = y.to_numpy()
    y = y.reshape(len(y), 1)
    prev_cost = None
    bol = True
    for j in range(iterations):
        crnt_cost = costFunction(x,y,w)
        if prev_cost is not None and abs(prev_cost - crnt_cost) <= stoping_threshold:
            break
        m = len(x)
        prev_cost = crnt_cost
        mat = np.matmul(x,w)
        u = 1/(1 + np.exp((-1)*np.clip(mat, -500, 500))).astype(np.longdouble)
        u = [1 if i >= 0.5 else 0 for i in u]
        u = np.array(u)
        u = u.reshape(len(u), 1)
        derivative = np.subtract(u, y)
        n = len(w) 
        for i in range(n):
            derv = np.multiply(derivative,x[:,i].reshape(m,1))
            w[i,0] = w[i,0] - learning_rate*sum(derv)
        b = b - learning_rate*sum(derivative)
    print(j)
    return w, b

In [13]:
y = trainData['Class']

In [14]:
x = trainData.drop(columns = ['Class'])

In [15]:
w = np.ones(len(x.columns))
w = w.reshape(len(x.columns),1)

In [16]:
b, b0 = gradient(x,y,w,1,0.00001)

9999


In [17]:
print(b, b0)

[[ 3.67858645]
 [-1.8570587 ]
 [12.685167  ]
 [ 1.820613  ]
 [ 0.99812143]
 [ 0.86905092]
 [ 0.78879916]
 [ 0.92167091]
 [ 0.99598802]
 [ 1.00580403]
 [ 1.00730536]
 [ 0.95960119]
 [ 0.34404608]
 [-6.32820859]
 [ 0.99716677]
 [ 0.96187759]
 [ 0.94780601]
 [ 0.98912342]
 [ 0.99022497]
 [ 0.99727459]
 [ 3.79678264]
 [-4.0382662 ]
 [ 8.2338532 ]
 [-3.461905  ]
 [ 0.97662237]
 [ 0.49706528]
 [ 0.36117622]
 [ 0.84578841]
 [ 0.89631528]
 [ 0.96885182]] [1.35664]


In [18]:
yTest = testData['Class']

In [19]:
xTest = testData.drop(columns = ['Class'])

In [20]:
b = b.ravel()

In [21]:
yOutput = []
b = b.ravel()
for d in xTest.iterrows():
    out = sum(np.multiply(d[1].to_numpy(), b)) + b0[0]
    out = 1/(1 + np.exp((-1)*np.clip(out, -500, 500)))
    if out >= 0.5:
        out = 1
    else:
        out = 0
    yOutput.append((out, yTest[d[0]]))

In [22]:
def confusion():
    confmat = np.array([0,0,0,0])
    confmat = confmat.reshape(2,2)
    for _, d in testData.iterrows():
        y = d['Class']
        x = d.drop('Class')
        out = sum(np.multiply(x,b) + b0[0])
        predy = 1/(1 + np.exp((-1)*np.clip(out, -500, 500)))
        if predy >= 0.5:
            predy = 1
        else:
            predy = 0
        if y == predy:
            if y == 0:
                confmat[0][0] += 1
            else:
                confmat[1][1] += 1
        else:
            if y == 0:
                confmat[1][0] += 1
            else:
                confmat[0][1] += 1
    return confmat

In [23]:
confusion()

array([[33,  0],
       [11, 70]])

In [24]:
def accuracy(mat):
    return (mat[0][0] + mat[1][1])/(mat[0][0] + mat[0][1] + mat[1][0] + mat[1][1])
def precision(mat):
    return (mat[0][0])/(mat[0][0] + mat[1][0])
def recall(mat):
    return (mat[0][0])/(mat[0][0] + mat[0][1])
def F1Score(p,r):
    return (2*(p*r))/(p + r)

In [25]:
mat = confusion() 
acc = accuracy(mat)
pre = precision(mat)
rec = recall(mat)
fscore = F1Score(pre, rec)
print(acc, pre, rec, fscore)

0.9035087719298246 0.75 1.0 0.8571428571428571
